**Упражнение 1.**
Перед вами фрагмент кода, содержащего некоторую проблему. Всегда ли counter = 10 после запуска программы?

In [3]:
import threading
import sys

def thread_job():
    global counter
    old_counter = counter
    counter = old_counter + 1
    print('{} '.format(counter), end='')
    sys.stdout.flush()


counter = 0
threads = [threading.Thread(target=thread_job) for _ in range(10)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()
print(counter)

1 2 3 4 5 6 7 8 9 10 10


Для наглядности продемонстрируем "проблему"

In [4]:
import threading
import random
import time
import sys

def thread_job():
    global counter
    old_counter = counter
    time.sleep(random.randint(0, 1))
    counter = old_counter + 1
    print('{} '.format(counter), end='')
    sys.stdout.flush()


counter = 0
threads = [threading.Thread(target=thread_job) for _ in range(10)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()
print(counter)

1 2 3 4 5 1 1 1 1 1 1


**Объясните почему так происходит?**



Причина наблюдаемой проблемы лежит в возникновении так называемого "race condition" (*состояния гонки*), т.е. такого состояния, когда результат выполнения программы зависит от порядка выполнения частей программы. 

В данном случае несколько потоков имеют общий ресурс (переменная `counter`), над которым они выполняют неатомарную операцию инкремента. Из-за этого в программе может сложиться ситуация, когда два или более потоков могут прочесть одно и то же значение переменной `counter`, прибавить к нему `1`, после чего записать полученное значение обратно в `counter`, что приводит к нарушению ожидаемой логики работы программы.

Решением данной проблемы может стать использование мьютексов. В языке Python мьютексы представлены объектом `threading.Lock`, который позволяет останавливать резервировать определенный ресурс до окончания выполнения операций над ним и предотвратить доступ к нему из других частей программы.

**Исправьте проблему.**


In [11]:
import threading, random, time, sys

from threading import Lock

counter_lock = Lock()

def thread_job():
    global counter
    with counter_lock:
        old_counter = counter
        time.sleep(random.randint(0, 1))
        counter = old_counter + 1
    print('{} '.format(counter), end='')
    sys.stdout.flush()


counter = 0
threads = [threading.Thread(target=thread_job) for _ in range(10)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()
print(counter)

1 2 3 4 5 6 7 8 9 10 10


**Упражнение 2.**
Программист хочет узнать доступность набора ip адресов. Он реализовал программу. Почему она неэффективна? Переделайте с использованием threading. Измерить время с применением потоков и без них. Объяснить результат.

In [ ]:
import os, re

received_packages = re.compile(r"(\d) received")

def status(x):
    if x == 0:
        return "no response"
    elif x == 1:
        return "losses"
    elif x == 2:
        return "alive"

time0 = time.time()

for suffix in range(0, 100):
    ip = "192.168.178." + str(suffix)
    ping_out = os.popen("ping -q -c2 " + ip, "r")  # получение вердикта
    #print("... pinging ", ip)
    while True:
        line = ping_out.readline()
        if not line:
            break
        n_received = received_packages.findall(line)
        #if n_received:
    print("Status: ", ip, status(-1))

print(f"time: ", time.time() - time0)

**Оптимизированная реализация:**

In [1]:
import os
import re
import time
import threading

print_lock = threading.Lock()

received_packages = re.compile(r"(\d) received")


def status(x):
    if x == '0':
        return "no response"
    elif x == '1':
        return "losses"
    elif x == '2':
        return "alive"

# этот код некорректно выполнится на Google Colab (в нем нет команды ping)
def ping(address):
    ping_out = os.popen("ping -q -c2 " + address, "r")
    while line := ping_out.readline():
        matches = received_packages.findall(line)
        if len(matches) > 0:
            n_received = matches[0]
    with print_lock:
        print("Status: ", address, status(n_received))


time0 = time.time()

addresses = [f'192.168.178.{suffix}' for suffix in range(0, 100)]

threads = [threading.Thread(target=ping, args=(address, ))
           for address in addresses]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

print(f"time: ", time.time() - time0)

Status:  192.168.178.14 no response
Status:  192.168.178.17 no response
Status:  192.168.178.12 no response
Status:  192.168.178.1 no response
Status:  192.168.178.4 no response
Status:  192.168.178.15 no response
Status:  192.168.178.8 no response
Status:  192.168.178.24 no response
Status:  192.168.178.5 no response
Status:  192.168.178.7 no response
Status:  192.168.178.18 no response
Status:  192.168.178.13 no response
Status:  192.168.178.22 no response
Status:  192.168.178.10 no response
Status:  192.168.178.23 no response
Status:  192.168.178.16 no response
Status:  192.168.178.9 no response
Status:  192.168.178.25 no response
Status:  192.168.178.21 no response
Status:  192.168.178.2 no response
Status:  192.168.178.6 no response
Status:  192.168.178.0 no response
Status:  192.168.178.20 no response
Status:  192.168.178.19 no response
Status:  192.168.178.3 no response
Status:  192.168.178.11 no response
Status:  192.168.178.46 no response
Status:  192.168.178.30 no response
St

**Объясните:**


Данная программа выполняет последовательность из 100 задач, каждую из которых можно разделить на две части: выполнение сетевого запроса и запись результата в консоль. Каждый сетевой запрос занимает значительное количество времени (~11с), поэтому суммарное время выполнения 100 задач составляет примерно 1100c. 

Данную программу можно оптимизировать, сделав выполнение задач параллельным. При этом важно помнить, что задачи требуют использование общего ресурса (стандартного потока вывода), поэтому следует ограничить доступ к нему, например, с помощью объекта `threading.Lock`.

|Способ|Время (c)|
|:-|:-:|
|Последовательное выполнение|1100.97|
|Параллельное выполнение|11.175|

**Упражнение 3.**
Составить программу, которая считает сумму элементов массива (создать из K значений и заполнить случайным образом) с использованием N потоков. Запустить с разным параметром N (2, 4, 8, 16, 32, 64). Объяснить результат (потребуется измерить время).

In [7]:
from random import uniform
from threading import Thread, Lock

def sum_array_multithread(arr, num_threads):
    arr_sum = 0
    arr_sum_lock = Lock()

    def task(task_num):
        nonlocal arr_sum
        local_sum = sum(arr[task_num::num_threads])
        with arr_sum_lock:
            arr_sum = arr_sum + local_sum

    threads = [Thread(target=task, args=(i, )) for i in range(0, num_threads)]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

    return arr_sum

In [8]:
from random import uniform

k = 1024 * 1024 * 8
arr = [uniform(0, 100) for _ in range(0, k)]

In [9]:
%timeit sum_array_multithread(arr, 2)
%timeit sum_array_multithread(arr, 4)
%timeit sum_array_multithread(arr, 8)
%timeit sum_array_multithread(arr, 16)
%timeit sum_array_multithread(arr, 32)
%timeit sum_array_multithread(arr, 64)

680 ms ± 71.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
871 ms ± 95.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.09 s ± 9.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.45 s ± 30.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.52 s ± 336 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.67 s ± 197 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Объяснение:**

Время работы программы растет с ростом количества потоков. Это можно объяснить тем фактом, что потоки не дают прироста в скорости вычислений, так как не могут выполняться одновременно (из-за GIL), при этом создание потоков является затратной операцией, что негативно влияет на время исполнения программы.

**Упражнение 4.** Запустите на исполнение следующий фрагмент кода, замерив время работы. Перепишите с помощью потоков и опять замерьте время. Объясните результат.

In [ ]:
import urllib.request
import time


urls = [
    'https://www.yandex.ru', 'https://www.google.com',
    'https://habrahabr.ru', 'https://www.python.org',
    'https://isocpp.org',
]


def read_url(url):
    with urllib.request.urlopen(url) as u:
        return u.read()


start = time.time()
for url in urls:
    read_url(url)
print(time.time() - start)

3.8154046535491943


**Оптимизированная программа:**

In [ ]:
import urllib.request
import time
from threading import Thread

urls = [
    'https://www.yandex.ru', 'https://www.google.com',
    'https://habrahabr.ru', 'https://www.python.org',
    'https://isocpp.org',
]


def read_url(url):
    with urllib.request.urlopen(url) as u:
        return u.read()


start = time.time()

threads = [Thread(target=read_url, args=(url, )) for url in urls]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

print(time.time() - start)

1.1073436737060547


**Результат:**

|Программа|Время (с)|
|-|:-:|
|Исходная|3,8|
|Оптимизированная|1,1|

**Объяснение:**

Уменьшение времени выполнения оптимизированной программы происходит по причине того, что в ней использован механизм многопоточного выполнения задач, позволяющий производить несколько действий параллельно, тем самым экономя время в сравнении с исходной реализацией, которая выполняет задачи последовательно.

**Упражнение 5.**
Составить программу, которая имеет общие ресурсы для нескольких потоков. Например, есть общая переменная, один поток добавляет 1, второй увеличивает значение в 2 раза. Написать с использованием Lock. Продемонстрировать проблему взаимной блокировки. Исправить её, написав код с использованием RLock блокировки.

In [ ]:
# эта программа никогда не выполнится
from threading import Thread, Lock

value = 0
lock = Lock()


def increment():
    global value
    with lock:
        value += 1


def double():
    global value
    with lock:
        saved = value
        while value - saved != saved:
            increment()


threads = [
    Thread(target=increment),
    Thread(target=double)
]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

print(value)


KeyboardInterrupt: 

**Объяснение:**

Код выше попадает в deadlock когда второй поток вызывает функцию `increment()`, так как в ней ожидается освобождение ресурса, занятого самим вторым потоком. Данная проблема решается использованием `threading.RLock`, который в отличие от `threading.Lock` позволяет повторно блокировать ресурс, если первая блокировка произошла в этом же потоке.

In [ ]:
from threading import Thread, RLock

value = 0
lock = RLock()


def increment():
    global value
    with lock:
        value += 1


def double():
    global value
    with lock:
        saved = value
        while value - saved != saved:
            increment()


threads = [
    Thread(target=increment),
    Thread(target=double)
]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

print(value)

2
